In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pickle
import os
import tensorflow as tf
import keras
from keras import layers

In [ ]:
# change the file path before running it
def load_ae_data(train_file='/Users/admin/Desktop/Code/data/ae.train', test_file='/Users/admin/Desktop/Code/data/ae.test'):
    """
    Load and parse the ape call data from ASCII files.
    
    Returns:
    - train_inputs: list of numpy arrays (270 time series, each N x 12)
    - test_inputs: list of numpy arrays (370 time series, each N x 12) 
    - train_outputs: list of numpy arrays (270 time series, each N x 9)
    - test_outputs: list of numpy arrays (370 time series, each N x 9)
    - N is the length of the time series, varying per sample
    """
    
    # Load the data
    ae_train = np.loadtxt(train_file)
    ae_test = np.loadtxt(test_file)
    
    # Parse training data
    train_inputs = []
    read_index = 0
    
    for c in range(270):
        start_index = read_index
        # Find the length of current time series (until we hit a row of 1.0s)
        while read_index < len(ae_train) and ae_train[read_index, 0] != 1.0:
            read_index += 1
        
        # Extract the time series
        time_series = ae_train[start_index:read_index, :]
        train_inputs.append(time_series)
        
        # Skip the separator row of 1.0s
        read_index += 1
    
    # Parse test data
    test_inputs = []
    read_index = 0
    
    for c in range(370):
        start_index = read_index
        # Find the length of current time series
        while read_index < len(ae_test) and ae_test[read_index, 0] != 1.0:
            read_index += 1
        
        # Extract the time series
        time_series = ae_test[start_index:read_index, :]
        test_inputs.append(time_series)
        
        # Skip the separator row of 1.0s
        read_index += 1
    
    # Generate teacher signals (outputs)
    train_outputs = []
    for c in range(270):
        length = len(train_inputs[c])
        teacher = np.zeros((length, 9))
        speaker_index = c // 30  # 30 samples per speaker, 9 speakers total
        teacher[:, speaker_index] = 1
        train_outputs.append(teacher)
    
    test_outputs = []
    speaker_index = 0
    block_counter = 0
    block_lengths = [31, 35, 88, 44, 29, 24, 40, 50, 29]  # samples per speaker in test set
    
    for c in range(370):
        if block_counter == block_lengths[speaker_index]:
            speaker_index += 1
            block_counter = 0
        
        length = len(test_inputs[c])
        teacher = np.zeros((length, 9))
        teacher[:, speaker_index] = 1
        test_outputs.append(teacher)
        
        block_counter += 1
    
    return train_inputs, test_inputs, train_outputs, test_outputs

In [ ]:
# Create data directory if it doesn't exist
os.makedirs('/Users/admin/Desktop/Code/data', exist_ok=True)

# Load the data
train_inputs, test_inputs, train_outputs, test_outputs = load_ae_data()

# Save the data as separate pickle files
with open('/Users/admin/Desktop/Code/data/train_inputs.pkl', 'wb') as f:
    pickle.dump(train_inputs, f)
with open('/Users/admin/Desktop/Code/data/test_inputs.pkl', 'wb') as f:
    pickle.dump(test_inputs, f)
with open('/Users/admin/Desktop/Code/data/train_outputs.pkl', 'wb') as f:
    pickle.dump(train_outputs, f)
with open('/Users/admin/Desktop/Code/data/test_outputs.pkl', 'wb') as f:
    pickle.dump(test_outputs, f)

In [ ]:
def load_saved_data(data_folder='/Users/admin/Desktop/Code/data'):
    """
    Load the preprocessed ape call data from pickle files.
    
    Args:
        data_folder (str): Path to the folder containing pickle files
        
    Returns:
        tuple: (train_inputs, test_inputs, train_outputs, test_outputs)
    """
    try:
        with open(f'{data_folder}/train_inputs.pkl', 'rb') as f:
            train_inputs = pickle.load(f)
        with open(f'{data_folder}/test_inputs.pkl', 'rb') as f:
            test_inputs = pickle.load(f)
        with open(f'{data_folder}/train_outputs.pkl', 'rb') as f:
            train_outputs = pickle.load(f)
        with open(f'{data_folder}/test_outputs.pkl', 'rb') as f:
            test_outputs = pickle.load(f)
        
        print(f"Data loaded successfully from {data_folder}/")
        print(f"Train samples: {len(train_inputs)}, Test samples: {len(test_inputs)}")
        print(f"Input dimensions: {train_inputs[0].shape[1]}, Output dimensions: {train_outputs[0].shape[1]}")
        
        return train_inputs, test_inputs, train_outputs, test_outputs
        
    except FileNotFoundError as e:
        print(f"Error: {e}")
        return None, None, None, None
    except Exception as e:
        print(f"Error loading data: {e}")
        return None, None, None, None

In [ ]:
train_inputs, test_inputs, train_outputs, test_outputs = load_saved_data()

In [ ]:
#get the maximum length for the trainning data set
def max_Len(input_data):
    length = [i.shape[0] for i in input_data]
    return int(max(length))

In [ ]:
#padding
def padding_to_maxl(input_data,max_l):
    padding_data = []
    for i in input_data:
        Time_step, LPC = i.shape
        if Time_step >= max_l:
            padding = i[:max_l]
        else:
            padding = np.pad(i, ((0,max_l-Time_step),(0,0)), mode="constant", constant_values=0.0)
        padding_data.append(padding)
    return np.stack(padding_data,axis = 0)

In [ ]:
Length = max_Len(train_inputs)
padding_train_data = padding_to_maxl(train_inputs, Length)
padding_test_data = padding_to_maxl(test_inputs, Length)

print(f"padding train input: {padding_train_data.shape}")
print(f"padding test input: {padding_test_data.shape}")